In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
import warnings
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.model_selection import train_test_split 

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
charging_set = pd.read_csv("data/charging_sessions.csv")

In [67]:
# general information about dataset, only use if needed for better overview of notebook
# TODO: implement it for the assignment!!!!!
# charging_set.info()
# charging_set.describe()
# charging_set.columns

charging_set.head(10)

,Unnamed: 0,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,userInputs,connectionTime_date,connectionTime_time,disconnectTime_date,disconnectTime_time,doneChargingTime_date,doneChargingTime_time,chargingDuration
0,0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1_1_179_810_2020-01-02 13:08:53.870034,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'mil...",2020-01-02,13:08:54,2020-01-02,19:11:15,2020-01-02,17:31:35,0 days 06:02:21
1,1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1_1_193_825_2020-01-02 13:36:49.599853,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,"[{'WhPerMile': 280, 'kWhRequested': 70.0, 'mil...",2020-01-02,13:36:50,2020-01-02,22:38:21,2020-01-02,20:18:05,0 days 09:01:31
2,2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1_1_193_829_2020-01-02 13:56:35.214993,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile...",2020-01-02,13:56:35,2020-01-03,00:39:22,2020-01-02,16:35:06,0 days 10:42:47
3,3,5e23b149f9af8b5fe4b973d2,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1_1_193_820_2020-01-02 13:59:58.309319,1,AG-1F04,1-1-193-820,America/Los_Angeles,1117.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile...",2020-01-02,13:59:58,2020-01-02,16:38:39,2020-01-02,15:18:45,0 days 02:38:41
4,4,5e23b149f9af8b5fe4b973d3,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,2020-01-02 18:17:30+00:00,13.375,1_1_193_819_2020-01-02 14:00:00.779967,1,AG-1F06,1-1-193-819,America/Los_Angeles,334.0,"[{'WhPerMile': 400, 'kWhRequested': 16.0, 'mil...",2020-01-02,14:00:01,2020-01-02,22:08:40,2020-01-02,18:17:30,0 days 08:08:39
5,5,5e23b149f9af8b5fe4b973d4,2020-01-02 14:00:13+00:00,2020-01-02 23:00:41+00:00,2020-01-02 20:13:21+00:00,13.370,1_1_194_821_2020-01-02 14:00:05.241573,1,AG-1F13,1-1-194-821,America/Los_Angeles,3519.0,"[{'WhPerMile': 600, 'kWhRequested': 24.0, 'mil...",2020-01-02,14:00:13,2020-01-02,23:00:41,2020-01-02,20:13:21,0 days 09:00:28
6,6,5e23b149f9af8b5fe4b973d5,2020-01-02 14:09:14+00:00,2020-01-03 01:57:58+00:00,2020-01-03 00:59:21+00:00,43.477,1_1_178_817_2020-01-02 14:09:14.451827,1,AG-1F09,1-1-178-817,America/Los_Angeles,933.0,"[{'WhPerMile': 385, 'kWhRequested': 65.45, 'mi...",2020-01-02,14:09:14,2020-01-03,01:57:58,2020-01-03,00:59:21,0 days 11:48:44
7,7,5e23b149f9af8b5fe4b973d6,2020-01-02 14:17:32+00:00,2020-01-03 00:44:54+00:00,2020-01-02 20:22:40+00:00,32.441,1_1_191_804_2020-01-02 14:17:32.329535,1,AG-4F34,1-1-191-804,America/Los_Angeles,3737.0,"[{'WhPerMile': 300, 'kWhRequested': 45.0, 'mil...",2020-01-02,14:17:32,2020-01-03,00:44:54,2020-01-02,20:22:40,0 days 10:27:22
8,8,5e23b149f9af8b5fe4b973d7,2020-01-02 14:25:38+00:00,2020-01-02 19:36:05+00:00,2020-01-02 18:38:39+00:00,13.264,1_1_194_826_2020-01-02 14:25:37.578692,1,AG-1F11,1-1-194-826,America/Los_Angeles,419.0,"[{'WhPerMile': 400, 'kWhRequested': 40.0, 'mil...",2020-01-02,14:25:38,2020-01-02,19:36:05,2020-01-02,18:38:39,0 days 05:10:27
9,9,5e23b149f9af8b5fe4b973d8,2020-01-02 14:27:40+00:00,2020-01-02 20:01:48+00:00,2020-01-02 19:17:46+00:00,13.285,1_1_178_823_2020-01-02 14:27:39.582337,1,AG-1F08,1-1-178-823,America/Los_Angeles,651.0,"[{'WhPerMile': 400, 'kWhRequested': 20.0, 'mil...",2020-01-02,14:27:40,2020-01-02,20:01:48,2020-01-02,19:17:46,0 days 05:34:08


# Time Data Preparation/Feature Engineering (draft)

For better data understanding and to improve data accessibility and interpretability, it is important to convert the current time data format into a human-readable time datatype. Currently the time related columns ("connectionTime", "disconnectionTime", "doneChargingTime") are formatted in the UTC format. To prepare the data for further interpretation we need to gather the relevant information. In this case the important information is the charging duration.

To get the duration we divide each of the time related columns into separate date and datetime columns. With these columns, the difference between connecting the EV with the station ("connectionTime") and disconnecting the EV with the station ("disconnectTime") can be calculated and stored in a separate column ("chargingDuration").


In [69]:
### split connectionTime in separate day and time columns
# Convert the column to datetime format if it's not already
charging_set['connectionTime'] = pd.to_datetime(charging_set['connectionTime'])

# Create a new column for the date part
charging_set['connectionTime_date'] = charging_set['connectionTime'].dt.date

# Create a new column for the time part
charging_set['connectionTime_time'] = charging_set['connectionTime'].dt.time


### split disconnectTime in separate day and time columns
# Convert the column to datetime format if it's not already
charging_set['disconnectTime'] = pd.to_datetime(charging_set['disconnectTime'])

# Create a new column for the date part
charging_set['disconnectTime_date'] = charging_set['disconnectTime'].dt.date

# Create a new column for the time part
charging_set['disconnectTime_time'] = charging_set['disconnectTime'].dt.time


### split doneChargingTime in separate day and time columns
# Convert the column to datetime format if it's not already
charging_set['doneChargingTime'] = pd.to_datetime(charging_set['doneChargingTime'])

# Create a new column for the date part
charging_set['doneChargingTime_date'] = charging_set['doneChargingTime'].dt.date

# Create a new column for the time part
charging_set['doneChargingTime_time'] = charging_set['doneChargingTime'].dt.time

charging_set_time

,Unnamed: 0,id,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,userInputs,connectionTime_date,connectionTime_time,disconnectTime_date,disconnectTime_time,doneChargingTime_date,doneChargingTime_time,chargingDuration
0,0,5e23b149f9af8b5fe4b973cf,25.016,1_1_179_810_2020-01-02 13:08:53.870034,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'mil...",2020-01-02,13:08:54,2020-01-02,19:11:15,2020-01-02,17:31:35,0 days 06:02:21
1,1,5e23b149f9af8b5fe4b973d0,33.097,1_1_193_825_2020-01-02 13:36:49.599853,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,"[{'WhPerMile': 280, 'kWhRequested': 70.0, 'mil...",2020-01-02,13:36:50,2020-01-02,22:38:21,2020-01-02,20:18:05,0 days 09:01:31
2,2,5e23b149f9af8b5fe4b973d1,6.521,1_1_193_829_2020-01-02 13:56:35.214993,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile...",2020-01-02,13:56:35,2020-01-03,00:39:22,2020-01-02,16:35:06,0 days 10:42:47
3,3,5e23b149f9af8b5fe4b973d2,2.355,1_1_193_820_2020-01-02 13:59:58.309319,1,AG-1F04,1-1-193-820,America/Los_Angeles,1117.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile...",2020-01-02,13:59:58,2020-01-02,16:38:39,2020-01-02,15:18:45,0 days 02:38:41
4,4,5e23b149f9af8b5fe4b973d3,13.375,1_1_193_819_2020-01-02 14:00:00.779967,1,AG-1F06,1-1-193-819,America/Los_Angeles,334.0,"[{'WhPerMile': 400, 'kWhRequested': 16.0, 'mil...",2020-01-02,14:00:01,2020-01-02,22:08:40,2020-01-02,18:17:30,0 days 08:08:39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66445,10083,5d574ad2f9af8b4c10c03652,28.787,1_1_179_809_2019-07-31 18:08:04.432654,1,AG-3F27,1-1-179-809,America/Los_Angeles,393.0,"[{'WhPerMile': 240, 'kWhRequested': 31.2, 'mil...",2019-07-31,18:08:04,2019-07-31,23:29:18,2019-07-31,23:30:18,0 days 05:21:14
66446,10084,5d574ad2f9af8b4c10c03653,7.787,1_1_179_810_2019-07-31 18:40:40.900203,1,AG-3F30,1-1-179-810,America/Los_Angeles,220.0,"[{'WhPerMile': 333, 'kWhRequested': 6.66, 'mil...",2019-07-31,18:40:41,2019-08-01,00:59:42,2019-07-31,21:44:23,0 days 06:19:01
66447,10085,5d574ad2f9af8b4c10c03654,11.274,1_1_191_795_2019-07-31 19:04:40.098273,1,AG-4F51,1-1-191-795,America/Los_Angeles,1974.0,"[{'WhPerMile': 333, 'kWhRequested': 19.98, 'mi...",2019-07-31,19:04:40,2019-07-31,22:44:22,2019-07-31,22:45:21,0 days 03:39:42
66448,10086,5d574ad2f9af8b4c10c03655,11.589,1_1_191_778_2019-07-31 19:19:46.919358,1,AG-4F43,1-1-191-778,America/Los_Angeles,942.0,"[{'WhPerMile': 275, 'kWhRequested': 22.0, 'mil...",2019-07-31,19:19:47,2019-08-01,00:34:51,2019-07-31,21:25:30,0 days 05:15:04


In [60]:
##### calculate charging duration
charging_set["chargingDuration"] = (charging_set["disconnectTime"] - charging_set["connectionTime"])

# clean "connectionTime", "disconnectTime", "doneChargingTime" from dataframe
charging_set_time = charging_set.drop(columns=["connectionTime", "disconnectTime", "doneChargingTime"])

charging_set_time
# TODO: convert current "chargingDuration" format (d days hh:mm:ss) into total hours e.g. 6.5 hours (0 days 06:30:00)

,Unnamed: 0,id,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,userInputs,connectionTime_date,connectionTime_time,disconnectTime_date,disconnectTime_time,doneChargingTime_date,doneChargingTime_time,chargingDuration
0,0,5e23b149f9af8b5fe4b973cf,25.016,1_1_179_810_2020-01-02 13:08:53.870034,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'mil...",2020-01-02,13:08:54,2020-01-02,19:11:15,2020-01-02,17:31:35,0 days 06:02:21
1,1,5e23b149f9af8b5fe4b973d0,33.097,1_1_193_825_2020-01-02 13:36:49.599853,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,"[{'WhPerMile': 280, 'kWhRequested': 70.0, 'mil...",2020-01-02,13:36:50,2020-01-02,22:38:21,2020-01-02,20:18:05,0 days 09:01:31
2,2,5e23b149f9af8b5fe4b973d1,6.521,1_1_193_829_2020-01-02 13:56:35.214993,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile...",2020-01-02,13:56:35,2020-01-03,00:39:22,2020-01-02,16:35:06,0 days 10:42:47
3,3,5e23b149f9af8b5fe4b973d2,2.355,1_1_193_820_2020-01-02 13:59:58.309319,1,AG-1F04,1-1-193-820,America/Los_Angeles,1117.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile...",2020-01-02,13:59:58,2020-01-02,16:38:39,2020-01-02,15:18:45,0 days 02:38:41
4,4,5e23b149f9af8b5fe4b973d3,13.375,1_1_193_819_2020-01-02 14:00:00.779967,1,AG-1F06,1-1-193-819,America/Los_Angeles,334.0,"[{'WhPerMile': 400, 'kWhRequested': 16.0, 'mil...",2020-01-02,14:00:01,2020-01-02,22:08:40,2020-01-02,18:17:30,0 days 08:08:39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66445,10083,5d574ad2f9af8b4c10c03652,28.787,1_1_179_809_2019-07-31 18:08:04.432654,1,AG-3F27,1-1-179-809,America/Los_Angeles,393.0,"[{'WhPerMile': 240, 'kWhRequested': 31.2, 'mil...",2019-07-31,18:08:04,2019-07-31,23:29:18,2019-07-31,23:30:18,0 days 05:21:14
66446,10084,5d574ad2f9af8b4c10c03653,7.787,1_1_179_810_2019-07-31 18:40:40.900203,1,AG-3F30,1-1-179-810,America/Los_Angeles,220.0,"[{'WhPerMile': 333, 'kWhRequested': 6.66, 'mil...",2019-07-31,18:40:41,2019-08-01,00:59:42,2019-07-31,21:44:23,0 days 06:19:01
66447,10085,5d574ad2f9af8b4c10c03654,11.274,1_1_191_795_2019-07-31 19:04:40.098273,1,AG-4F51,1-1-191-795,America/Los_Angeles,1974.0,"[{'WhPerMile': 333, 'kWhRequested': 19.98, 'mi...",2019-07-31,19:04:40,2019-07-31,22:44:22,2019-07-31,22:45:21,0 days 03:39:42
66448,10086,5d574ad2f9af8b4c10c03655,11.589,1_1_191_778_2019-07-31 19:19:46.919358,1,AG-4F43,1-1-191-778,America/Los_Angeles,942.0,"[{'WhPerMile': 275, 'kWhRequested': 22.0, 'mil...",2019-07-31,19:19:47,2019-08-01,00:34:51,2019-07-31,21:25:30,0 days 05:15:04


In [45]:
charging_set.columns

Index(['Unnamed: 0', 'id', 'connectionTime', 'disconnectTime',
       'doneChargingTime', 'kWhDelivered', 'sessionID', 'siteID', 'spaceID',
       'stationID', 'timezone', 'userID', 'userInputs', 'connectionTime_date',
       'connectionTime_time', 'disconnectTime_date', 'disconnectTime_time',
       'doneChargingTime_date', 'doneChargingTime_time'],
      dtype='object')

In [47]:
##### check for wrong values e.g. negative values, unrealistic values
# print("id data range:", np.sort(charging_set["id"].unique()))
print("connectionTime data range:", np.sort(charging_set["connectionTime"].unique()))
print("disconnectTime data range:", np.sort(charging_set["disconnectTime"].unique()))
print("doneChargingTime data range:", np.sort(charging_set["doneChargingTime"].unique()))
print("kWhDelivered data range:", np.sort(charging_set["kWhDelivered"].unique()))
# print("sessionID data range:", np.sort(charging_set["sessionID"].unique()))
# print("siteID data range:", np.sort(charging_set["siteID"].unique()))
# print("spaceID data range:", np.sort(charging_set["spaceID"].unique()))
# print("stationID data range:", np.sort(charging_set["stationID"].unique()))
print("timezone data range:", np.sort(charging_set["timezone"].unique()))
# print("userID data range:", np.sort(charging_set["userID"].unique()))
# print("userInputs data range:", np.sort(charging_set["userInputs"].unique()))
print("connectionTime_date data range:", np.sort(charging_set["connectionTime_date"].unique()))
print("connectionTime_time data range:", np.sort(charging_set["connectionTime_time"].unique()))
print("disconnectTime_date data range:", np.sort(charging_set["disconnectTime_date"].unique()))
print("disconnectTime_time data range:", np.sort(charging_set["disconnectTime_time"].unique()))
# print("doneChargingTime_date data range:", np.sort(charging_set["doneChargingTime_date"].unique()))
# print("doneChargingTime_time data range:", np.sort(charging_set["doneChargingTime_time"].unique()))



connectionTime data range: [Timestamp('2018-04-25 11:08:04+0000', tz='UTC')
 Timestamp('2018-04-25 13:45:10+0000', tz='UTC')
 Timestamp('2018-04-25 13:45:50+0000', tz='UTC') ...
 Timestamp('2021-09-14 01:08:16+0000', tz='UTC')
 Timestamp('2021-09-14 01:52:37+0000', tz='UTC')
 Timestamp('2021-09-14 05:43:39+0000', tz='UTC')]
disconnectTime data range: [Timestamp('2018-04-25 13:20:10+0000', tz='UTC')
 Timestamp('2018-04-25 18:15:46+0000', tz='UTC')
 Timestamp('2018-04-25 18:27:51+0000', tz='UTC') ...
 Timestamp('2021-09-14 03:05:10+0000', tz='UTC')
 Timestamp('2021-09-14 03:17:55+0000', tz='UTC')
 Timestamp('2021-09-14 14:46:28+0000', tz='UTC')]
doneChargingTime data range: [Timestamp('2018-04-25 13:21:10+0000', tz='UTC')
 Timestamp('2018-04-25 14:51:44+0000', tz='UTC')
 Timestamp('2018-04-25 16:05:22+0000', tz='UTC') ...
 Timestamp('2021-09-14 01:31:19+0000', tz='UTC')
 Timestamp('2021-09-14 01:34:15+0000', tz='UTC')
 Timestamp('2021-09-14 14:46:22+0000', tz='UTC')]
kWhDelivered data ra

In [61]:
### check column "doneChargingTime" for wrong datatype
# hypothesis: all float values in this column are nan-values
# Find the indices of float values in a column that should contain only strings
# float_indices = [idx for idx, value in charging_set['doneChargingTime'].items() if isinstance(value, float) and not isinstance(value, str)]

# Print the result
# if float_indices:
#     print("Indices of float values in column 'doneChargingTime' (all others should be strings):", float_indices)
# else:
#     print("No float values found in the column.")

In [63]:
# Get indices where 'column_name' has NaN values
# nan_indices = charging_set[charging_set['doneChargingTime'].isna()].index.tolist()

# Print the indices of NaN values
#  print(nan_indices)

In [10]:
# check for duplicate rows 

duplicate_counter = charging_set.duplicated().sum()
duplicates = charging_set[charging_set.duplicated()]
print(duplicate_counter)
print(duplicates)

0
Empty DataFrame
Columns: [Unnamed: 0, id, connectionTime, disconnectTime, doneChargingTime, kWhDelivered, sessionID, siteID, spaceID, stationID, timezone, userID, userInputs]
Index: []
